This is a notebook that can be used to pull data from the iRacing API.

https://forums.iracing.com/discussion/15068/general-availability-of-data-api/p1

uses the [iracingdataapi package](https://github.com/jasondilworth56/iracingdataapi) to pull data from the iRacing API.



In [ ]:
!pip install git+https://github.com/jasondilworth56/iracingdataapi.git


In [26]:
from iracingdataapi.client import irDataClient

import os

Race = 5

username = os.getenv("IRACING_USERNAME")
password = os.getenv("IRACING_PASSWORD")


# alternatively you can set the username and password directly here
# fill in your username and password, an uncomment the lines below
# username = "putyourusernamehereitisusuallyyouremailaddress"
# password = "putyourpasswordhere"

def calculate_f499_points(start, finish, incident_count):
    return (start - finish) + (4 if incident_count == 0 else -1 * (2 * incident_count))


def get_session_link(subsession_id, new_ui=False):
    if new_ui:
        format = "https://members-ng.iracing.com/racing/results-stats/results?subsessionid="
    else:
        format = "https://members.iracing.com/membersite/member/EventResult.do?&subsessionid="

    return f"{format}{subsession_id}"


def last_10_results(client: irDataClient, customer_id=None):
    # if you do not pass a customer_id as the param to the method call, it will default to the customer_id of the user you are logged in as
    # note: this API call returns only the last 10 races, I think.
    stats = client.stats_member_recent_races(customer_id)
    # get the stats for the last 10 races
    last_10_races = stats['races']
    # iterate over the last 10 races
    for i, race in enumerate(last_10_races, start=1):
        start_time = race['session_start_time']
        subsession_id = race['subsession_id']
        start_position = race['start_position']
        finish_position = race['finish_position']
        incident_count = race['incidents']
        series_name = race['series_name']
        track_name = race['track']['track_name']
        points = calculate_f499_points(start_position, finish_position, incident_count)

        print_race_results(subsession_id, start_time, start_position, finish_position, incident_count, points,
                           series_name, track_name)
    print(f"Total Races: {len(last_10_races)}")


def print_race_results(subsession_id, start_time, start_position, finish_position, incident_count, points, series_name,
                       track_name):
    print(f"Race {start_time}:")
    print(f"Session Link: [{subsession_id}]({get_session_link(subsession_id, True)})")
    print(f"Series: {series_name}")
    print(f"Track: {track_name}")
    print(f"Start Position: {start_position if start_position else 'N/A'}")
    print(f"Finish Position: {finish_position if finish_position else 'N/A'}")
    print(f"Number of Incidents: {incident_count}")
    print(f"F499 Scoring Points: {points}")
    print()


def get_499_series(client: irDataClient):
    series = client.series
    # print out the series
    # find the series with the string "LMP3" in the name
    _499_series = [s for s in series if "LMP3" in s['series_name']]
    # # add any series that has the string "LMP2" in the name to the list
    # _499_series.extend([s for s in series if "LMP2" in s['series_name']])
    # # add any series that has the string "F4" in the name to the list
    # _499_series.extend([s for s in series if "FIA Formula 4" in s['series_name']])
    # # add any series that has the string "Falken Sports Car" in the name to the list
    # _499_series.extend([s for s in series if "Falken Tyre Sports Car" in s['series_name']])
    # add any series that has the string "IMSA" in the name to the list
    _499_series.extend([s for s in series if "IMSA iRacing Series" in s['series_name']])

    # put the series_id and series_name in a tuple and add them to a list
    _499_series = [(s['series_id'], s['series_name']) for s in _499_series]
    return _499_series


def run():
    cust_id = 929408  # Hannah
    # cust_id = 643506  # Tom
    cust_id = 818734  # JD
    idc = irDataClient(username=username, password=password)
    # last_10_results(idc, cust_id)
    series_of_interest = get_499_series(idc)
    
    print(f"Customer ID: {cust_id}, 2024S3 Week 1")
    for series_id, series_name in series_of_interest:
        series_results = idc.result_search_series(cust_id=cust_id, series_id=series_id, official_only=True,
                                                  event_types=[Race], season_year=2024, season_quarter=3)

        print(f"Series: {series_name}")
        print(f"Series ID: {series_id}")
        print(f"{len(series_results)} Races")
        print("=" * 36)
        # print the series results
        for i, result in enumerate(series_results, start=1):
            start_position = result['starting_position_in_class'] + 1
            finish_position = result['finish_position_in_class'] + 1
            incident_count = result['incidents']
            points = calculate_f499_points(start_position, finish_position, incident_count)

            print_race_results(result['subsession_id'], result['start_time'], start_position, finish_position,
                               incident_count, points, series_name, result['track']['track_name'])
            print("-" * 36)

        print("=" * 36)
        print()


run()


# seasons = idc.season_list(season_year=2024, season_quarter=2)
# season_id = seasons[0]['season_id']
# more_results = idc.result_season_results(season_id, race_week_num=1)



Customer ID: 818734, 2024S3 Week 1
Series: LMP3 Trophy - Fixed
Series ID: 525
1 Races
Race 2024-06-13T18:30:00Z:
Session Link: [69424553](https://members-ng.iracing.com/racing/results-stats/results?subsessionid=69424553)
Series: LMP3 Trophy - Fixed
Track: Motorsport Arena Oschersleben
Start Position: 10
Finish Position: 13
Number of Incidents: 0
F499 Scoring Points: 1

------------------------------------

Series: IMSA iRacing Series - Fixed
Series ID: 539
1 Races
Race 2024-06-14T15:45:00Z:
Session Link: [69444092](https://members-ng.iracing.com/racing/results-stats/results?subsessionid=69444092)
Series: IMSA iRacing Series - Fixed
Track: Watkins Glen International
Start Position: 10
Finish Position: 6
Number of Incidents: 0
F499 Scoring Points: 8

------------------------------------

Series: IMSA iRacing Series
Series ID: 447
1 Races
Race 2024-06-15T16:45:00Z:
Session Link: [69467359](https://members-ng.iracing.com/racing/results-stats/results?subsessionid=69467359)
Series: IMSA iRac